In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import cv2
from pycocotools.coco import COCO
from PIL import Image
import torch
import os
import matplotlib.pyplot as plt
#import colormap as cmap
import numpy as np


def visualize(index, masks, bounding_boxes):
    image = coco.loadImgs(coco.getImgIds()[index])[0]    
    anno_ids = coco.getAnnIds(imgIds=[image["id"]])
    annos = coco.loadAnns(anno_ids) 
    
    """new_annos = []
    for single in annos:
        single = get_angles(single)
        new_annos.append([single])
    
    annos = new_annos"""
    filename = os.path.join(data_folder,image["file_name"])
    img = cv2.imread(str(filename))
    print(img.shape)
    for anno in annos:
        #print(anno)
        anno = get_angles(anno)
        #c = tuple([int(x) for x in anno["category_id"]])         
        if masks:
            points = np.array(anno["segmentation"], dtype=np.int32).reshape(1,-1,2)
            img = cv2.polylines(img, points, True, 3)
        
        if bounding_boxes:  
                x,y,w,h,a = np.array(anno["bbox"]).astype(np.int32)
                #img = cv2.rectangle(img, (x,y),(x + w,y + h),(0,255,255), 3) 
                img = draw_xywha(img,x,y,w,h,a)
                cat = coco.cats[anno['category_id']]['name']
                cv2.putText(img, cat, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX ,  0.8, 1, cv2.LINE_AA)            
                
    
    plt.imshow(img[...,::-1])
    plt.axis("off")
    plt.grid(False)
    plt.tight_layout()

def get_angles(ann):
    
    #print(ann['bbox'])
    img_id = ann['image_id']
    new_ann = None
    # get width and height
    if not 'rbbox' in ann:
        # using COCO dataset. 4 = [x1,y1,w,h]
        coco = True
        # convert COCO format: x1,y1,w,h to x,y,w,h
        ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
        ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
        ann['bbox'].append(0)
        if ann['bbox'][2] > ann['bbox'][3]:
            ann['bbox'][2],ann['bbox'][3] = ann['bbox'][3],ann['bbox'][2]
            ann['bbox'][4] -= 90
        new_ann = ann['bbox']
    else:
        # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
        assert len(ann['rbbox']) == 5, 'Unknown bbox format' # x,y,w,h,a
        new_ann = ann['rbbox']
        

        if new_ann[2] > new_ann[3]:
            new_ann[2], new_ann[3] = new_ann[3],new_ann[2]
            new_ann[4] -= 90 if new_ann[4] > 0 else -90

    if new_ann[2] == new_ann[3]:
        new_ann[3] += 1 # force that w < h
        
    new_ann[4] = np.clip(new_ann[4], -90.0, 90.0 - 1e-14)

    assert new_ann[2] < new_ann[3]
    assert new_ann[4] >= -90 and new_ann[4] < 90

    # override original bounding box with rotated bounding box
    ann['bbox'] = torch.Tensor(new_ann)
    #print(ann['bbox'])
    return ann

    
def draw_xywha(im, x, y, w, h, angle, color=(255,0,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w/2, -h/2], [w/2, -h/2], [w/2, h/2], [-w/2, h/2]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([x, y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    return im
    
    
#data_folder = "/localdata/saurabh/dataset/FES/JPEGImages/"
#coco = COCO("/localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json")
data_folder = "/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/JPEGImages/"
coco = COCO("/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/instances_training.json")
plt.figure(figsize=(100,100))
total = len(coco.getImgIds()) - 1

interact(visualize, index=widgets.IntSlider(min=0, max=total, step=1, continuous_update=False), masks=False, bounding_boxes=True);  
plt.show()

# Make the data files

In [1]:
import glob
from collections import defaultdict
import os
import json
import numpy as np
import cv2
from PIL import Image
import torch
image_paths = glob.glob("/localdata/saurabh/yolov3/data/custom/images/train/*img.png")


image_ids = []
image_names = []
#print(image_paths[0])
for image in image_paths:
    #print(image)
    image_name = os.path.splitext(os.path.basename(image))[0]
    #print(image_name)
    image_names += [image_name]
    image_name = image_name.split('_')[0]
    image_ids += [image_name]
    #break
    
#print(image_ids)

In [2]:
len(image_ids)

49700

In [2]:
sorted(image_names)

[]

In [4]:
def load_anns(img_dir,json_path):
        coco = False
        print(f'Loading annotations {json_path} into memory...')
        with open(json_path, 'r') as f:
            json_data = json.load(f)

        for ann in json_data['annotations']:
            img_id = ann['image_id']
            new_ann = None
            # get width and height 
            if not 'rbbox' in ann:
                # using COCO dataset. 4 = [x1,y1,w,h]
                coco = True
                # convert COCO format: x1,y1,w,h to x,y,w,h
                ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
                ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
                ann['bbox'].append(0)
                if ann['bbox'][2] > ann['bbox'][3]:
                    ann['bbox'][2], ann['bbox'][3] = ann['bbox'][3], ann['bbox'][2]
                    ann['bbox'][4] -= 90
                new_ann = ann['bbox']
            else:
                # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
                # x,y,w,h,a
                assert len(ann['rbbox']) == 5, 'Unknown bbox format'
                new_ann = ann['rbbox']

                if new_ann[2] > new_ann[3]:
                    new_ann[2], new_ann[3] = new_ann[3], new_ann[2]
                    new_ann[4] += 90 - np.finfo(np.float32).eps

            if new_ann[2] == new_ann[3]:
                new_ann[3] += 1  # force that w < h

            assert new_ann[2] < new_ann[3]
            assert new_ann[4] >= -90 and new_ann[4] < 90

            # override original bounding box with rotated bounding box
            ann['bbox'] = torch.Tensor(new_ann)
            imgid2anns[img_id].append(ann)

        for img in json_data['images']:
            img_id = img['id']
            assert img_id not in imgid2path
            anns = imgid2anns[img_id]
            # if there is crowd gt, skip this image
            if coco and any(ann['iscrowd'] for ann in anns):
                continue

            img_ids.append(img_id)
            imgid2path[img_id] = os.path.join(img_dir, img['file_name'])
            imgid2info[img['id']] = img

        catids = [cat['id'] for cat in json_data['categories']]

In [5]:
coco =  False
img_ids = []
imgid2info = dict()
imgid2path = dict()
imgid2anns = defaultdict(list)
catids = []

load_anns("/localdata/saurabh/dataset/theodore_v3/images/", "/localdata/saurabh/dataset/theodore_v3/coco/annotations/instances.json" )

Loading annotations /localdata/saurabh/dataset/theodore_v3/coco/annotations/instances.json into memory...


In [ ]:
image_ids = []
for i in range(len(imgid2info)):
    file_name = imgid2info[i+1]['file_name'].split('.')[0]
    if file_name in image_names:
        image_ids += [i+1]

In [6]:
len(image_ids)

49700

In [7]:
ind_path = [imgid2path[int(ids)] for ids in image_ids]
annotations = [imgid2anns[int(ids)] for ids in image_ids]

In [20]:
image_names

['0049796_img',
 '0049807_img',
 '0049785_img',
 '0049894_img',
 '0049773_img',
 '0049829_img',
 '0049813_img',
 '0049989_img',
 '0049760_img',
 '0049926_img',
 '0049904_img',
 '0049964_img',
 '0049879_img',
 '0049899_img',
 '0049997_img',
 '0049782_img',
 '0049967_img',
 '0049784_img',
 '0049788_img',
 '0049728_img',
 '0049700_img',
 '0049811_img',
 '0049988_img',
 '0049724_img',
 '0049716_img',
 '0049898_img',
 '0049779_img',
 '0049775_img',
 '0049897_img',
 '0049783_img',
 '0049889_img',
 '0049824_img',
 '0049993_img',
 '0049980_img',
 '0049780_img',
 '0049827_img',
 '0049723_img',
 '0049790_img',
 '0049888_img',
 '0049890_img',
 '0049804_img',
 '0049800_img',
 '0049973_img',
 '0049852_img',
 '0049885_img',
 '0049859_img',
 '0049805_img',
 '0049913_img',
 '0049842_img',
 '0049831_img',
 '0049914_img',
 '0049972_img',
 '0049726_img',
 '0049968_img',
 '0049939_img',
 '0049738_img',
 '0049802_img',
 '0049962_img',
 '0049876_img',
 '0049941_img',
 '0049755_img',
 '0049976_img',
 '004992

In [8]:
label_map = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5}

In [9]:
label_map[3]

2

In [10]:
all_images = []
#text_names = open('/localdata/saurabh/yolov3/data/custom/train_paths.txt', "w")
for i, ann in enumerate(annotations):
    file_name = image_names[i]
    #append the path list
    text_names.write('/localdata/saurabh/yolov3/data/custom/images/train/{}.png'.format(file_name))
    text_names.write('\n')
    #open the label file for each anno
    #text_file = open('/localdata/saurabh/yolov3/data/custom/labels/train/{}.txt'.format(file_name), "w") 
    for anno in ann:
        #print(anno['category_id'])
        text_file.write("%i " %label_map[anno['category_id']])
        text_file.writelines(["%f " % item  for item in anno['bbox']])
        text_file.write("\n")
    
    text_file.close()
    #break
text_names.close()

print(file_name)
    


0026169_img


In [ ]:
annotations

In [ ]:
with open('/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/captions_training.json', 'r') as f:
            json_data = json.load(f)

In [ ]:
json_data['images']